# Building A RNN From Scratch

Import the modules.

In [1]:
import numpy as np
from collections import Counter
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.models import Sequential
from keras.utils import to_categorical
from keras.preprocessing.text import one_hot
from keras.layers.recurrent import SimpleRNN
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


Let's consider an example text: "This is an example."

Define the input and output.

In [2]:
#define documents
docs = ['this is','is an']
# define class labels
labels = ['an','example']

The strategy that we'll adopt to build an RNN is as follows:

* 1. One-hot encode the words.
* 2. Identify the maximum length of input:Pad the rest of input to the maximum length so that all the inputs are of the same length.
* 3. Convert the words in the input into a one-hot-encoded version.
* 4. Convert the words in the output into a one-hot-encoded version.
* 5. Process the input and the output data, then fit the RNN model.

Preprocess our dataset so that it can be passed to an RNN.

In the code 
* Identifying all the unique words and their corresponding frequency (counts) in the dataset. 
* Assigning an ID number to each word.

In [3]:
counts = Counter()
for i,review in enumerate(docs+labels):
    counts.update(review.split())
words = sorted(counts, key=counts.get, reverse=True)
vocab_size=len(words)
word_to_int = {word: i for i, word in enumerate(words, 1)}

print word_to_int.

In [4]:
word_to_int

{'is': 1, 'an': 2, 'this': 3, 'example': 4}

Modify the input and output words with their corresponding IDs.

* Appending the ID of each word of an input sentence into a list, thus making the input (encoded_docs) a list of lists.
* Appending the ID of each word of the output into a list.

In [5]:
encoded_docs = []
for doc in docs:
    encoded_docs.append([word_to_int[word] for word in doc.split()])
encoded_labels = []
for label in labels:
    encoded_labels.append([word_to_int[word] for word in label.split()])

In [6]:
print('encoded_docs: ',encoded_docs)
print('encoded_labels: ',encoded_labels)

encoded_docs:  [[3, 1], [1, 2]]
encoded_labels:  [[2], [4]]


We also have to take care of input length while encoding the input. 
* In cases of sentiment analysis, the input text length can vary from one review to another. 
* The neural network expects the input size to be fixed. 
* To tackle this problem, we perform padding on top of the input. 
* Padding ensures that all inputs are encoded to have a similar length. 

In the code, we are passing encoded_docs to the pad_sequences function, which ensures that all the input data points have the same length—which is equal to the maxlen parameter.

# For those parameters that are shorter than maxlen 
* It pads data points with a value of 0 to achieve a total length of maxlen and the zero padding is done pre so it will add pads value of 0, to the left of the original encoded sentence.

In [7]:
# pad documents to a max length of 2 words
max_length = 2
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='pre')
print(padded_docs)

[[3 1]
 [1 2]]


* Now that the input dataset is created, let's preprocess the output dataset so that it can be passed to the model-training step.
* The typical processing for outputs is to make them into dummy values, that is, make a one-hot-encoded version of the output labels.

In [8]:
# processing the output dataset
one_hot_encoded_labels = to_categorical(encoded_labels, num_classes=5)
print(one_hot_encoded_labels)

[[0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]]


Note: Given the output values (encoded_labels) are {2, 4}, the output vectors have a value of 1 at the second and fourth positions respectively.

# Let's build the model.

An RNN expects the input to be (batch_size, time_steps, and features_per_timestep) in shape. Hence, we first reshape the padded_docs input into the following format.

In [9]:
padded_docs = padded_docs.reshape(2,2,1)

* The 'return_sequences' parameter specifies whether we want to obtain the hidden layer values at each time step. A false value for 'return_sequences' specifies that we want the hidden layer output only at the final timestep.

* In a many-to-one task, where there are many inputs (one input in each time step) and outputs, 'return_sequences' will be false, resulting in the output being obtained only in the final time step. An example of this could be the stock price on the next day, given a sequence of historical five-day stock prices.

* In cases where we try to obtain hidden-layer values in each time step, 'return_sequences' will be set to True. An example of this could be machine translation, where there are many inputs and many outputs.

In [10]:
# define the model
embed_length=1
max_length=2
model = Sequential()
model.add(SimpleRNN(1,activation='tanh', return_sequences=False,recurrent_initializer='Zeros',input_shape=(max_length,embed_length),unroll=True))

Instructions for updating:
Colocations handled automatically by placer.


Connect the RNN output to five nodes of the output layer.

* We have performed a Dense(5), as there are five possible classes of output (the output of each example has 5 values, where each value corresponds to the probability of it belonging to word ID 0 to word ID 4).

In [11]:
model.add(Dense(5, activation='softmax'))

Compile and summarize the model.

In [12]:
# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 1)                 3         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 10        
Total params: 13
Trainable params: 13
Non-trainable params: 0
_________________________________________________________________
None


Fit the model to predict the output from the input.

In [13]:
model.fit(padded_docs,np.array(one_hot_encoded_labels),epochs=500)

Instructions for updating:
Use tf.cast instead.
Epoch 1/500
2/2 [==============================] - 1s 269ms/step - loss: 1.3483 - acc: 0.5000
Epoch 2/500
2/2 [==============================] - 0s 1ms/step - loss: 1.3464 - acc: 0.5000
Epoch 3/500
2/2 [==============================] - 0s 1ms/step - loss: 1.3447 - acc: 0.5000
Epoch 4/500
2/2 [==============================] - 0s 998us/step - loss: 1.3430 - acc: 0.5000
Epoch 5/500
2/2 [==============================] - 0s 2ms/step - loss: 1.3412 - acc: 0.5000
Epoch 6/500
2/2 [==============================] - 0s 1ms/step - loss: 1.3393 - acc: 0.5000
Epoch 7/500
2/2 [==============================] - 0s 996us/step - loss: 1.3375 - acc: 0.5000
Epoch 8/500
2/2 [==============================] - 0s 1ms/step - loss: 1.3356 - acc: 0.5000
Epoch 9/500
2/2 [==============================] - 0s 997us/step - loss: 1.3337 - acc: 0.5000
Epoch 10/500
2/2 [==============================] - 0s 997us/step - loss: 1.3319 - acc: 0.5000
Epoch 11/500
2/2 [===

Epoch 85/500
2/2 [==============================] - 0s 499us/step - loss: 1.2032 - acc: 0.5000
Epoch 86/500
2/2 [==============================] - 0s 997us/step - loss: 1.2017 - acc: 0.5000
Epoch 87/500
2/2 [==============================] - 0s 1ms/step - loss: 1.2002 - acc: 0.5000
Epoch 88/500
2/2 [==============================] - 0s 998us/step - loss: 1.1987 - acc: 0.5000
Epoch 89/500
2/2 [==============================] - 0s 997us/step - loss: 1.1972 - acc: 0.5000
Epoch 90/500
2/2 [==============================] - 0s 997us/step - loss: 1.1957 - acc: 0.5000
Epoch 91/500
2/2 [==============================] - 0s 1ms/step - loss: 1.1942 - acc: 0.5000
Epoch 92/500
2/2 [==============================] - 0s 998us/step - loss: 1.1928 - acc: 0.5000
Epoch 93/500
2/2 [==============================] - 0s 998us/step - loss: 1.1913 - acc: 0.5000
Epoch 94/500
2/2 [==============================] - 0s 997us/step - loss: 1.1898 - acc: 0.5000
Epoch 95/500
2/2 [==============================] - 0s

Epoch 171/500
2/2 [==============================] - 0s 1ms/step - loss: 1.0912 - acc: 0.5000
Epoch 172/500
2/2 [==============================] - 0s 1ms/step - loss: 1.0900 - acc: 0.5000
Epoch 173/500
2/2 [==============================] - 0s 1ms/step - loss: 1.0889 - acc: 0.5000
Epoch 174/500
2/2 [==============================] - 0s 998us/step - loss: 1.0878 - acc: 0.5000
Epoch 175/500
2/2 [==============================] - 0s 997us/step - loss: 1.0867 - acc: 0.5000
Epoch 176/500
2/2 [==============================] - 0s 998us/step - loss: 1.0856 - acc: 0.5000
Epoch 177/500
2/2 [==============================] - 0s 998us/step - loss: 1.0845 - acc: 0.5000
Epoch 178/500
2/2 [==============================] - 0s 997us/step - loss: 1.0834 - acc: 0.5000
Epoch 179/500
2/2 [==============================] - 0s 998us/step - loss: 1.0824 - acc: 0.5000
Epoch 180/500
2/2 [==============================] - 0s 998us/step - loss: 1.0813 - acc: 0.5000
Epoch 181/500
2/2 [===========================

Epoch 257/500
2/2 [==============================] - 0s 997us/step - loss: 1.0084 - acc: 0.5000
Epoch 258/500
2/2 [==============================] - 0s 499us/step - loss: 1.0076 - acc: 0.5000
Epoch 259/500
2/2 [==============================] - 0s 997us/step - loss: 1.0068 - acc: 0.5000
Epoch 260/500
2/2 [==============================] - 0s 993us/step - loss: 1.0059 - acc: 0.5000
Epoch 261/500
2/2 [==============================] - 0s 998us/step - loss: 1.0051 - acc: 0.5000
Epoch 262/500
2/2 [==============================] - 0s 998us/step - loss: 1.0043 - acc: 0.5000
Epoch 263/500
2/2 [==============================] - 0s 998us/step - loss: 1.0035 - acc: 0.5000
Epoch 264/500
2/2 [==============================] - 0s 1ms/step - loss: 1.0027 - acc: 0.5000
Epoch 265/500
2/2 [==============================] - 0s 1ms/step - loss: 1.0019 - acc: 0.5000
Epoch 266/500
2/2 [==============================] - 0s 997us/step - loss: 1.0011 - acc: 0.5000
Epoch 267/500
2/2 [=========================

Epoch 343/500
2/2 [==============================] - 0s 998us/step - loss: 0.9469 - acc: 0.5000
Epoch 344/500
2/2 [==============================] - 0s 998us/step - loss: 0.9463 - acc: 0.5000
Epoch 345/500
2/2 [==============================] - 0s 997us/step - loss: 0.9457 - acc: 0.5000
Epoch 346/500
2/2 [==============================] - 0s 997us/step - loss: 0.9451 - acc: 0.5000
Epoch 347/500
2/2 [==============================] - 0s 997us/step - loss: 0.9445 - acc: 0.5000
Epoch 348/500
2/2 [==============================] - 0s 499us/step - loss: 0.9439 - acc: 0.5000
Epoch 349/500
2/2 [==============================] - 0s 991us/step - loss: 0.9433 - acc: 0.5000
Epoch 350/500
2/2 [==============================] - 0s 1ms/step - loss: 0.9427 - acc: 0.5000
Epoch 351/500
2/2 [==============================] - 0s 997us/step - loss: 0.9421 - acc: 0.5000
Epoch 352/500
2/2 [==============================] - 0s 997us/step - loss: 0.9415 - acc: 0.5000
Epoch 353/500
2/2 [=======================

Epoch 429/500
2/2 [==============================] - 0s 499us/step - loss: 0.9008 - acc: 0.5000
Epoch 430/500
2/2 [==============================] - 0s 997us/step - loss: 0.9003 - acc: 0.5000
Epoch 431/500
2/2 [==============================] - 0s 997us/step - loss: 0.8998 - acc: 0.5000
Epoch 432/500
2/2 [==============================] - 0s 998us/step - loss: 0.8994 - acc: 0.5000
Epoch 433/500
2/2 [==============================] - 0s 997us/step - loss: 0.8989 - acc: 0.5000
Epoch 434/500
2/2 [==============================] - 0s 998us/step - loss: 0.8985 - acc: 0.5000
Epoch 435/500
2/2 [==============================] - 0s 997us/step - loss: 0.8980 - acc: 0.5000
Epoch 436/500
2/2 [==============================] - 0s 998us/step - loss: 0.8975 - acc: 0.5000
Epoch 437/500
2/2 [==============================] - 0s 997us/step - loss: 0.8971 - acc: 0.5000
Epoch 438/500
2/2 [==============================] - 0s 997us/step - loss: 0.8966 - acc: 0.5000
Epoch 439/500
2/2 [=====================

Extract prediction on the first input data point.

In [14]:
model.predict(padded_docs[0].reshape(1,2,1))

array([[0.06150892, 0.07722024, 0.42594436, 0.02466969, 0.41065678]],
      dtype=float32)

Inspect the weights.

In [15]:
model.weights

[<tf.Variable 'simple_rnn_1/kernel:0' shape=(1, 1) dtype=float32_ref>,
 <tf.Variable 'simple_rnn_1/recurrent_kernel:0' shape=(1, 1) dtype=float32_ref>,
 <tf.Variable 'simple_rnn_1/bias:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'dense_1/kernel:0' shape=(1, 5) dtype=float32_ref>,
 <tf.Variable 'dense_1/bias:0' shape=(5,) dtype=float32_ref>]

The above weights gives us an intuition of the order in which weights are presented in the output.

* The kernel represents the weights and recurrent represents the connection of the hidden layer from one step to another.

* A simpleRNN has weights that connect the input to the hidden layer and also weights that connect the previous time step's hidden layer to the current time step's hidden layer.

* The kernel and bias in the dense_1 layer represent the layer that connects the hidden layer value to the final output

The following line of code gives us the computed values of each of the weights.

In [16]:
model.get_weights()

[array([[1.9012635]], dtype=float32),
 array([[0.31144455]], dtype=float32),
 array([0.3112596], dtype=float32),
 array([[ 0.12269498,  0.35384947,  1.3085266 , -0.8057373 ,  1.2569675 ]],
       dtype=float32),
 array([-0.38604417, -0.38676906,  0.3783856 , -0.38306865,  0.3927357 ],
       dtype=float32)]

Pass the input through the first time step—the input is as follows.

In [17]:
padded_docs[0]

array([[3],
       [1]])

In the preceding code, the first time step has a value of 3 and the second time step has a value of 1. We'll initialize the value at first time step as follows.

In [18]:
input_t0 = 3

The value at the first time step is multiplied by the weight connecting the input to the hidden layer, then the bias value is added.

In [19]:
input_t0_kernel_bias = input_t0*model.get_weights()[0] + model.get_weights()[2]

The hidden layer value at this time step is calculated by passing the preceding output through the tanh activation (as that is the activation we specified when we defined the model).

In [20]:
hidden_layer0_value = np.tanh(input_t0_kernel_bias)

Calculate the hidden-layer value at time step 2; where the input has a value of 1 (note the value of padded_docs[0] is [3, 1])

In [21]:
input_t1 = 1

The output value when the input at the second time step is passed through the weight and bias is as follows
* Note: that the weights that multiply the input remain the same, regardless of the time step being considered

The final calculation of the hidden-layer value at a given time step would be the summation of the preceding two matrix multiplications. Pass the result through a tanh activation function.

In [22]:
input_t1_kernel_bias = input_t1*model.get_weights()[0] + model.get_weights()[2]

The calculation from the input layer to the hidden-layer constitutes two components

* Matrix multiplication of the input layer value and kernel weights.
* Matrix multiplication of the hidden layer of the previous time step and recurrent weights.

In [23]:
input_t1_recurrent = hidden_layer0_value*model.get_weights()[1]

The total value before passing through the tanh activation is as follows.

In [24]:
total_input_t1 = input_t1_kernel_bias + input_t1_recurrent

The output of the hidden-layer value is calculated by passing the preceding output through the tanh activation, as follows.

In [25]:
output_t1 = np.tanh(total_input_t1)

Pass the hidden layer output from the final time step through the dense layer, which connects the hidden layer to the output layer.
* Note that the fourth and fifth output of the model.get_weights() method correspond to the connection from the hidden layer to the output layer.

In [26]:
final_output = output_t1*model.get_weights()[3] + model.get_weights()[4]
final_output

array([[-0.26491517, -0.03743586,  1.6702111 , -1.1785221 ,  1.6336603 ]],
      dtype=float32)

Pass the preceding output through the softmax activation (as defined in the model) to obtain the final output.

In [27]:
np.exp(final_output)/np.sum(np.exp(final_output))

array([[0.06150893, 0.07722025, 0.42594433, 0.02466969, 0.4106568 ]],
      dtype=float32)

We can notice that the output we obtained through the forward pass of input through the network is the same as what the model.predict() function gave as a output.